# L5: Data Analyst Agent 🤓

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> , notebooks and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

First start by filtering warnings and loading environment variables.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from helper import load_env
load_env()

Now, let's create the data analyst agent. You will create a sandbox, then open the `pokemon.csv` dataset and write it to the sandbox as `data.csv`.

In [1]:
from lib.utils import create_sandbox

sbx = create_sandbox()

with open("pokemon.csv", "r") as f:
    content = f.read()

sbx.files.write("data.csv", content)

/Users/ytchen/Documents/experimental/Coding/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO     [sandbox] 🚀 Creating new Sandbox.create(id=if4k4mvodgmo9vos4vha6)

WriteInfo(name='data.csv', type='file', path='/home/user/data.csv')

Now you can import your `coding_agent` and modify the system prompt to let it know that the user has uploaded a dataset called `data.csv` and it should create interesting plots.

In [2]:
from lib.coding_agent import coding_agent, log
from lib.tools_schemas import execute_code_schema
from lib.tools import execute_code
from openai import OpenAI

client = OpenAI()

system = """You are a senior python programmer. 
You must run the code using the `execute_code` tool.
The user has uploaded a data.csv.
You help the user understanding the data 
by creating interesting plots.
"""

tools = { "execute_code" : execute_code }

Now you can query the data as normal and the sandbox will execute code on the data to answer the queries.

In [3]:
messages = []

query = "What is the data about?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"import pandas as pd\n\ndf = pd.read_csv('data.csv')\ndf.head(), df.columns, df.shape"}      │
│ Result: {"results": [{"text": "(                     abilities  against_bug  against_dark  against_dragon  \\\n │
│ 0  ['Overgrow', 'Chlorophyll']          1.0           1.0             1.0   \n 1  ['Overgrow', '...             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: The dataset contains information about different Pokémon. It has 801 rows and 41 columns. Some of the 
         columns include:                                                                                          
                                                                                                                   
         - abilities: The abilities of the Pokémon                                                                 
         - against_*: Various columns indicating how the Pokémon performs against different types (bug, dark,      
         dragon, etc.)                                                                                             
         - attack, defense, sp_attack, sp_defense, speed, hp: Combat stats of the Pokémon                          
         - base_egg_steps, base_happiness, base_total, capture_rate: Additional attributes related to Pokémon      
         breeding and catching                                                                                     
         - classification: The classification or category of the Pokémon                                           
         - height_m, weight_kg: Physical attributes of the Pokémon                                                 
         - name, japanese_name: The names of the Pokémon                                                           
         - type1, type2: Primary and secondary types of the Pokémon                                                
         - generation: The generation in which the Pokémon appears                                                 
         - is_legendary: Whether the Pokémon is legendary (1) or not (0)                                           
                                                                                                                   
         Would you like me to create some plots or analyze specific aspects of this data?

INFO     [agent] 🔢 tokens: 1299 total

In [4]:
query = "Can you aggregate the pokemons by type?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"# Aggregating Pokémon by their primary type\npoke_by_type =                                 │
│ df['type1'].value_counts().reset_index()\npoke_by_type.columns = ['type', 'count']\npoke_by_type"}              │
│ Result: {"results": [{"text": "        type  count\n0      water    114\n1     normal    105\n2      grass      │
│ 78\n3        bug     72\n4    psychic     53\n5       fire     52\n6       rock     45\n7   elect...            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: Here is the count of Pokémon aggregated by their primary type:                                        
                                                                                                                   
         - water: 114                                                                                              
         - normal: 105                                                                                             
         - grass: 78                                                                                               
         - bug: 72                                                                                                 
         - psychic: 53                                                                                             
         - fire: 52                                                                                                
         - rock: 45                                                                                                
         - electric: 39                                                                                            
         - poison: 32                                                                                              
         - ground: 32                                                                                              
         - dark: 29                                                                                                
         - fighting: 28                                                                                            
         - dragon: 27                                                                                              
         - ghost: 27                                                                                               
         - steel: 24                                                                                               
         - ice: 23                                                                                                 
         - fairy: 18                                                                                               
         - flying: 3                                                                                               
                                                                                                                   
         Would you like me to visualize this data in a plot?

INFO     [agent] 🔢 tokens: 2775 total

### Gradio UI

You can use this provided Gradio UI to have a nicer user experience when chatting with your code generation agent. It will also give interesting information about the context stack, including the number of tokens used in the result, by the assistant, the user, and tools.

In [5]:
from lib.ui import ui

ui(coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://01cfeedd2a39a78c99.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Your Turn

Now it's your turn to explore this unknown dataset. Ask questions and learn more about the information inside.

In [6]:
sbx = create_sandbox()
with open("unknown.csv", "rb") as f:
    content = f.read()
sbx.files.write("data.csv", content)

messages = []

ui(
    coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=if4k4mvodgmo9vos4vha6)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://b3e8ba16dee1194a54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
